In [ ]:
!pip -q install redis redis-cli

In [1]:
import redis
r = redis.Redis(host="127.0.0.1", port=6379, db=0)


## RedisJSON: Storing and Querying JSON Documents in Redis

### JSON Introduction

JSON (JavaScript Object Notation) is a lightweight data-interchange format that represents data objects as attribute–value pairs and arrays. It supports strings, numbers, booleans, null, objects, and arrays. These primitive values are also called JSON scalars.

Example JSON document:

```python
# Example JSON representation of a product
{
  "id": "B085LVV8R7",
  "name": "SENNHEISER Momentum True Wireless 2",
  "tags": ["hifi", "headphones", "wireless"]
}
```

> **Sidenote**
> [JSON Specification | ECMA](https://www.ecma-international.org/publications-and-standards/standards/ecma-404/)
> **Concept**: `JSON` → A standard format for representing structured data.
> **Context**: Widely used in APIs, databases, and configurations.
> **Example**: Objects with key-value pairs and arrays.
> **Implication**: Ensures interoperability and human-readable data exchange.

```python
# RedisJSON usage context
# Store JSON documents as Redis keys and access sub-elements efficiently.
```

### What is RedisJSON?

RedisJSON is a Redis module that implements JSON as a native data type. It allows storing, updating, and retrieving JSON values directly in Redis. Documents are stored as binary tree structures for fast access to sub-elements.

Key features include:

* Full support for JSON standard
* JSONPath syntax for selecting elements
* Atomic operations for all JSON value types
* Fast retrieval and updates on sub-elements

> **Sidenote**
> [RedisJSON | Docs - Redis](https://redis.io/docs/latest/develop/data-types/json/)
> **Concept**: `RedisJSON` → Redis module enabling JSON as a first-class data type.
> **Context**: Turns Redis into a document store.
> **Example**: Store and query structured product catalog entries.
> **Implication**: Enables document-oriented patterns on Redis.

```python
# Example JSON document for Redis storage
{
  "id": "B085LVV8R7",
  "name": "SENNHEISER Momentum True Wireless 2",
  "price": 150,
  "category": {
    "id": "wireless_headphones",
    "name": "Wireless headphones"
  },
  "tags": ["hifi", "headphones", "wireless"]
}
```

### Basic API with RedisJSON

#### Setting JSON Documents

```python
# Store a JSON product document
r.execute_command("JSON.SET", "product:1", "$", '{"id":"B085LVV8R7","name":"SENNHEISER Momentum True Wireless 2","price":150,"category":{"id":"wireless_headphones","name":"Wireless headphones"},"tags":["hifi","headphones","wireless"]}')
print(r.execute_command("JSON.GET", "product:1", "$"))
```

> **Sidenote**
> [JSON.SET | Docs - Redis](https://redis.io/docs/latest/commands/json.set/)
> **Command**: `JSON.SET` → Set a JSON value at a key.
> **Pattern**: `JSON.SET key path value`
> **Example**: `JSON.SET product:1 $ '{"name":"AirPods"}'`
> **Result**: Stores the document under the key.

#### Retrieving JSON Documents

```python
# Retrieve full JSON document
r.execute_command("JSON.GET", "product:1", "$")

# Retrieve only the product name
r.execute_command("JSON.GET", "product:1", "$.name")

# Retrieve nested category name
r.execute_command("JSON.GET", "product:1", "$.category.name")
```

> **Sidenote**
> [JSON.GET | Docs - Redis](https://redis.io/docs/latest/commands/json.get/)
> **Command**: `JSON.GET` → Retrieve JSON values.
> **Pattern**: `JSON.GET key [path]`
> **Example**: `JSON.GET product:1 $.name`
> **Result**: Returns the value of the JSON path.

#### Numeric Operations

```python
# Increase product price by 5
r.execute_command("JSON.NUMINCRBY", "product:1", "$.price", 5)
print(r.execute_command("JSON.GET", "product:1", "$.price"))

# Decrease product price by 5
r.execute_command("JSON.NUMINCRBY", "product:1", "$.price", -5)
print(r.execute_command("JSON.GET", "product:1", "$.price"))
```

> **Sidenote**
> [JSON.NUMINCRBY | Docs - Redis](https://redis.io/docs/latest/commands/json.numincrby/)
> **Command**: `JSON.NUMINCRBY` → Increment or decrement a numeric value.
> **Pattern**: `JSON.NUMINCRBY key path number`
> **Example**: `JSON.NUMINCRBY product:1 $.price 10`
> **Result**: Updates the numeric field by the given value.

#### Array Operations

```python
# Append new tags to product
r.execute_command("JSON.ARRAPPEND", "product:1", "$.tags", '"audio"', '"bluetooth"')
print(r.execute_command("JSON.GET", "product:1", "$.tags"))

# Count number of tags
r.execute_command("JSON.ARRLEN", "product:1", "$.tags")
```

> **Sidenote**
> [JSON.ARRAPPEND | Docs - Redis](https://redis.io/docs/latest/commands/json.arrappend/)
> **Command**: `JSON.ARRAPPEND` → Append values to an array.
> **Pattern**: `JSON.ARRAPPEND key path value [value ...]`
> **Example**: `JSON.ARRAPPEND product:1 $.tags '"newtag"'`
> **Result**: Adds new elements to array.

> **Sidenote**
> [JSON.ARRLEN | Docs - Redis](https://redis.io/docs/latest/commands/json.arrlen/)
> **Command**: `JSON.ARRLEN` → Get length of a JSON array.
> **Pattern**: `JSON.ARRLEN key path`
> **Example**: `JSON.ARRLEN product:1 $.tags`
> **Result**: Returns number of elements in array.

### JSON with RediSearch

RedisJSON integrates with RediSearch to provide indexing, full-text search, and query capabilities over JSON documents.

#### Creating an Index

```python
# Create an index for product documents
r.execute_command("FT.CREATE", "products", "ON", "JSON",
                  "PREFIX", "1", "product:",
                  "SCHEMA",
                  "$.id", "AS", "id", "TEXT",
                  "$.name", "AS", "name", "TEXT", "NOSTEM",
                  "$.price", "AS", "price", "NUMERIC", "SORTABLE",
                  "$.category.id", "AS", "category_id", "TEXT", "NOSTEM",
                  "$.category.name", "AS", "category_name", "TEXT",
                  "$.tags.*", "AS", "tags", "TAG")
print("Index created")
```

> **Sidenote**
> [FT.CREATE | Docs - Redis](https://redis.io/docs/latest/commands/ft.create/)
> **Command**: `FT.CREATE` → Create a RediSearch index.
> **Pattern**: `FT.CREATE index_name ON JSON PREFIX N schema`
> **Example**: `FT.CREATE products ON JSON PREFIX 1 "product:" SCHEMA $.id AS id TEXT`
> **Result**: Creates an index over JSON fields.

#### Searching with Indexes

```python
# Search by text across fields
r.execute_command("FT.SEARCH", "products", "headphones")

# Search by tag
r.execute_command("FT.SEARCH", "products", "@tags:{wireless}")

# Search by price range
r.execute_command("FT.SEARCH", "products", "@price:[120 150]")
```

> **Sidenote**
> [FT.SEARCH | Docs - Redis](https://redis.io/docs/latest/commands/ft.search/)
> **Command**: `FT.SEARCH` → Query an index.
> **Pattern**: `FT.SEARCH index query`
> **Example**: `FT.SEARCH products "@tags:{wireless}"`
> **Result**: Returns documents matching query.

#### Dropping an Index

```python
# Drop an index when no longer needed
r.execute_command("FT.DROPINDEX", "products")
print("Index dropped")
```

> **Sidenote**
> [FT.DROPINDEX | Docs - Redis](https://redis.io/docs/latest/commands/ft.dropindex/)
> **Command**: `FT.DROPINDEX` → Delete an index.
> **Pattern**: `FT.DROPINDEX index_name`
> **Example**: `FT.DROPINDEX products`
> **Result**: Removes the index definition.

---

## Sources

* [JSON Specification | ECMA](https://www.ecma-international.org/publications-and-standards/standards/ecma-404/)
* [RedisJSON | Docs - Redis](https://redis.io/docs/latest/develop/data-types/json/)
* [JSON.SET | Docs - Redis](https://redis.io/docs/latest/commands/json.set/)
* [JSON.GET | Docs - Redis](https://redis.io/docs/latest/commands/json.get/)
* [JSON.NUMINCRBY | Docs - Redis](https://redis.io/docs/latest/commands/json.numincrby/)
* [JSON.ARRAPPEND | Docs - Redis](https://redis.io/docs/latest/commands/json.arrappend/)
* [JSON.ARRLEN | Docs - Redis](https://redis.io/docs/latest/commands/json.arrlen/)
* [FT.CREATE | Docs - Redis](https://redis.io/docs/latest/commands/ft.create/)
* [FT.SEARCH | Docs - Redis](https://redis.io/docs/latest/commands/ft.search/)
* [FT.DROPINDEX | Docs - Redis](https://redis.io/docs/latest/commands/ft.dropindex/)




### Original Transcript

Article - JSON
JSON Introduction
JSON (JavaScript Object Notation) is an open standard file and data interchange format that uses human-readable text to store and transmit data objects.

## product:1

{

  "id": "B085LVV8R7",

  "name": "SENNHEISER Momentum True Wireless 2",

  "tags": ["hifi", "headphones", "wireless"]

}

In the example JSON above we can see that the data object consists of attribute–value pairs ("name":"SENNHEISER Momentum True Wireless 2") and arrays ("tags": ["hifi", "headphones", "wireless"]) or other serializable values. The attributes (id, name and tags) are always strings, while the values can be strings, numbers, booleans, null, objects or arrays. String, number, null or boolean values are also called JSON scalars.

What is RedisJSON?
RedisJSON is a Redis module that implements JSON as a native data type. It allows storing, updating and fetching JSON values from Redis keys (documents) which makes it a perfect fit for a document store.

Primary features include:

Full support of the JSON standard
JSONPath syntax for selecting elements inside documents
Documents are stored as binary data in a tree structure, allowing fast access to sub-elements
Typed atomic operations for all JSON values types
 

Basic API
The two commands you'll use the most with RedisJSON are the JSON.SET and JSON.GET; the former one to write JSON documents into Redis and the latter one to get data out of it. In the rest of this article we'll assume we're working with the following, simple JSON document:

# product:1

{

  "id": "B085LVV8R7",

  "name": "SENNHEISER Momentum True Wireless 2",

  "price": 150,

  "category": {

    "id": "wireless_headphones",

    "name": "Wireless headphones"

  },

  "tags": ["hifi", "headphones", "wireless"]

}

To load this document into Redis we can run:

JSON.SET product:1 $ '{"id":"B085LVV8R7","name":"SENNHEISER Momentum True Wireless 2","price":150,"category":{"id":"wireless_headphones","name":"Wireless headphones"},"tags":["hifi","headphones","wireless"]}'

JSON.SET product:2 $ '{"id":"B084H7DCW4","name":"Apple AirPods","price":129,"category":{"id":"wireless_headphones","name":"Wireless headphones"},"tags":["hifi","headphones","wireless"]}'

JSON.SET product:3 $ '{"id":"B08D2SGT4M","name":"Elgato Wave:3","price":152.99,"category":{"id":"usb_microphones","name":"USB microphones"},"tags":["usb","microphone","streaming","podcast"]}'

To read the whole document out we run:

JSON.GET product:1 $

To read only the name or the category name:

JSON.GET product:1 $.name

JSON.GET product:1 $.category.name

To increase or decrease the price:

JSON.NUMINCRBY product:1 $.price 5

JSON.NUMINCRBY product:1 $.price -5

To add a tag to the tags array:

JSON.ARRAPPEND product:1 $.tags '"audio"' '"bluetooth"'

To find the number of tags in the tags array:

JSON.ARRLEN product:1 $.tags

JSON and RediSearch
The RedisJSON module can be combined with the RediSearch module to index JSON documents in real-time. The indexes let us query data, perform aggregations, and filter by properties, numeric ranges, and geographical distance. By combining these two modules and adding native indexing, querying, and full-text search capabilities we expanded our multi-model story and we can now say that we support basic document database capabilities.

FT.CREATE products

 ON JSON 

    PREFIX 1 "product:"

 SCHEMA 

   $.id AS id TEXT 

   $.name AS name TEXT NOSTEM 

   $.price AS price NUMERIC SORTABLE 

   $.category.id AS category_id TEXT NOSTEM

   $.category.name AS category_name TEXT  

   $.tags.* AS tags TAG

Single-line version:

FT.CREATE products ON JSON PREFIX 1 "product:" SCHEMA $.id AS id TEXT $.name AS name TEXT NOSTEM $.price AS price NUMERIC SORTABLE $.category.id AS category_id TEXT NOSTEM $.category.name AS category_name TEXT $.tags.* AS tags TAG

Search for items including a determined text across all the text fields:

FT.SEARCH products "headphones"

Result: Every item with the desired search term included in a TEXT field

Search for items tagged with a determined tag:

FT.SEARCH products "@tags:{wireless}"

Result: Every item with classified with the desired tag

Search for items whose price is included in the desired range

FT.SEARCH products "@price:[120,150]"

Result: Every item with the desired search term included in a TEXT field

Drop the index using the following command:

FT.DROPINDEX products